In [245]:
import pandas as pd
import numpy as np

In [246]:
df = pd.read_csv('2. Monthly.csv')

In [247]:
df.head()

,Gender,Monthly,Donation Source,Category,Unit,2005,2006,2007,2008,2009,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,month
0,Men+Women,January,Total,Donation Count,Unit,169087.0,176015.0,187538.0,201135.0,190237.0,...,257355.0,248426.0,219578.0,237893.0,238712.0,206758.0,213118.0,189598.0,200795.0,1
1,Men+Women,January,Total,Composition Rate[%],%,7.4,7.6,9.0,8.6,7.4,...,8.3,8.7,7.5,8.3,8.6,7.9,8.2,7.2,7.2,1
2,Men+Women,January,Korean Red Cross,Donation Count,Unit,165863.0,173132.0,182945.0,195331.0,182670.0,...,241385.0,231909.0,203609.0,221066.0,223703.0,194040.0,200754.0,175710.0,185559.0,1
3,Men+Women,January,Korean Red Cross,Composition Rate[%],%,7.5,7.7,9.0,8.6,7.4,...,8.4,8.8,7.5,8.2,8.6,8.0,8.3,7.2,7.3,1
4,Men+Women,January,Central Seoul,Donation Count,Unit,15577.0,17563.0,17631.0,21047.0,16614.0,...,NaN,NaN,NaN,24809.0,23563.0,19831.0,17856.0,16737.0,17767.0,1


In [248]:
# remove column that has unnamed in it

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [249]:
df.columns

Index(['Gender', 'Monthly', 'Donation Source', 'Category', 'Unit', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       'month'],
      dtype='object')

In [250]:
policy_df = pd.read_csv('policies.csv')

cities = policy_df['city'].unique()

In [251]:
print(cities)

['Seoul' nan 'Daejeon/Chungnam' 'Gyeongnam' 'Jeju' 'Daegu/Gyeongbuk'
 'Ulsan' 'Gangwon' 'Incheon' 'Gyeonggi' 'Busan' 'Chungbuk' 'Jeonbuk'
 'Jeonnam']


In [252]:
df['Donation Source'].unique()


array(['Total', 'Korean Red Cross', 'Central Seoul', 'Western Seoul',
       'Southern Seoul', 'Eastern Seoul', 'Busan', 'Daegu/Gyeongbuk',
       'Incheon', 'Ulsan', 'Gyeonggi', 'Gangwon', 'Chungbuk',
       'Daejeon/Chungnam', 'Jeonbuk', 'Jeonnam', 'Gyeongnam', 'Jeju',
       'Outside of Korean Red Cross'], dtype=object)

In [253]:
set(cities) - set(df['Donation Source'].unique())

# save the data to a csv file
df.to_csv('2. Monthly.csv', index=False)

In [254]:
# delete the rows that have column "Monthly" as "Total"
df = df[df['Monthly'] != 'Total']

In [255]:

years = [str(i) for i in range(2006, 2024)]

months = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

# create a month column in the df 
df['month'] = df['Monthly'].map(months)

print(years)

print(df["month"].isnull().sum())


['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
0


In [256]:
total_dict = {} # the key will be the year_month_city
women_dict = {}
men_dict = {}

for idx, row in df.iterrows():
    gender = row["Gender"]
    city = row['Donation Source']
    month = row['month']
    category = row['Category']

    # all other rows are the year
    for year in years:
        key = None
        if category == "Donation Count":
            key = f"{year}_{month}_{city}_unit"
        elif category == "Composition Rate[%]":
            key = f"{year}_{month}_{city}_rate"

        if key is None:
            raise Exception("Key is None")
        
        val = row[year]

        if gender == "Men+Women":
            # it means total
            total_dict[key] = val
            continue

        elif gender == "Men":
            # it means men
            men_dict[key] = val

        
        elif gender == "Women":
            # it means women
            women_dict[key] = val
        
        else:
            raise Exception("Unknown gender", gender)



In [257]:
print(len(total_dict))
print(len(men_dict))
print(len(women_dict))

8208
8208
8208


In [258]:
material = {
    "city": [],
    "year": [],
    "month": [],
    "composition_rate": [],
    "total_unit": [],
    "women_unit": [],
    "men_unit": []
}

# now we can create a data frame

for key in total_dict:
    year, month, city, unit = key.split("_")
    val = total_dict[key]

    if unit == "unit":
        material["city"].append(city)
        material["year"].append(year)
        material["month"].append(month)
        material["total_unit"].append(val)
    elif unit == "rate":
        material["composition_rate"].append(val)
    else:
        raise Exception("Unknown unit", unit)
    
for key in men_dict:
    city, year, month, unit = key.split("_")
    val = men_dict[key]

    if unit == "unit":
        material["men_unit"].append(val)

for key in women_dict:
    city, year, month, unit = key.split("_")
    val = women_dict[key]

    if unit == "unit":
        material["women_unit"].append(val)
    

In [259]:
for key in material:
    print(key, len(material[key]))

city 4104
year 4104
month 4104
composition_rate 4104
total_unit 4104
women_unit 4104
men_unit 4104


In [260]:
# make it to a dataframe
final_df = pd.DataFrame(material)

# sort by city, year and month
final_df = final_df.sort_values(by=['city', 'year', 'month'])

# Remove columns that are not included in the cities

final_df = final_df[~final_df['city'].isin(["Korean Red Cross", "Outside of Korean Red Cross", "Total"])]

final_df = final_df.replace("-", np.nan)

# fill in the nan values with 0
final_df = final_df.fillna(0)



In [261]:
print(final_df["city"].unique())

['Busan' 'Central Seoul' 'Chungbuk' 'Daegu/Gyeongbuk' 'Daejeon/Chungnam'
 'Eastern Seoul' 'Gangwon' 'Gyeonggi' 'Gyeongnam' 'Incheon' 'Jeju'
 'Jeonbuk' 'Jeonnam' 'Southern Seoul' 'Ulsan' 'Western Seoul']


In [262]:
final_df[final_df["city"] == "Western Seoul"]

,city,year,month,composition_rate,total_unit,women_unit,men_unit
54,Western Seoul,2006,1,6.7,9334.0,1534.0,7800.0
3132,Western Seoul,2006,10,8.3,11572.0,2370.0,9202.0
3474,Western Seoul,2006,11,8.6,11872.0,2384.0,9488.0
3816,Western Seoul,2006,12,10.1,13964.0,1978.0,11986.0
396,Western Seoul,2006,2,8.2,11299.0,2117.0,9182.0
...,...,...,...,...,...,...,...
1439,Western Seoul,2023,5,0,0,0,0
1781,Western Seoul,2023,6,0,0,0,0
2123,Western Seoul,2023,7,0,0,0,0
2465,Western Seoul,2023,8,0,0,0,0


In [263]:

# create new rows 

add_df = {
    "city": [],
    "year": [],
    "month": [],
    "composition_rate": [],
    "total_unit": [],
    "women_unit": [],
    "men_unit": []
}

# We are going to merge the values of Eastern Seoul, Western Seoul, Southern Seoul, Northern Seoul to Seoul

for year in years:
    for month in range(1, 13):
        total_unit = 0
        composition_rate = 0
        women_unit = 0
        men_unit = 0
        year = str(year)
        for city in final_df['city'].unique():
            # check if city contains Seoul
            if city.find("Seoul") != -1:
                city_data = final_df[final_df['city'] == city]
                city_data = city_data[city_data['year'] == str(year)]
                city_data = city_data[city_data['month'] == str(month)]
                # print(len(city_data))
                total_unit += float(city_data['total_unit'])
                composition_rate += float(city_data['composition_rate'])
                men_unit += float(city_data["men_unit"])
                women_unit += float(city_data["women_unit"])
            
        add_df['city'].append("Seoul")
        add_df['year'].append(year)
        add_df['month'].append(month)
        add_df['total_unit'].append(total_unit)
        add_df['composition_rate'].append(round(composition_rate, 2))
        add_df["men_unit"].append(men_unit)
        add_df["women_unit"].append(women_unit)



/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_42883/2813987408.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  total_unit += float(city_data['total_unit'])
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_42883/2813987408.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  composition_rate += float(city_data['composition_rate'])
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_42883/2813987408.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  men_unit += float(city_data["men_unit"])
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_42883/2813987408.py:32: FutureWarning: Calling float on a single element Series is deprecated and will raise a Typ

In [264]:
# append add_df to final_df
add_df = pd.DataFrame(add_df)

final_df = pd.concat([final_df, add_df])

In [265]:
len(final_df)

3672

In [266]:
final_df.describe()

,city,year,month,composition_rate,total_unit,women_unit,men_unit
count,3672,3672,3672,3672.0,3672,3672,3672
unique,17,18,24,208.0,3275,2886,3196
top,Busan,2006,1,8.7,0,0,0
freq,216,204,288,188.0,162,162,162


In [267]:
final_df["city"].unique()

array(['Busan', 'Central Seoul', 'Chungbuk', 'Daegu/Gyeongbuk',
       'Daejeon/Chungnam', 'Eastern Seoul', 'Gangwon', 'Gyeonggi',
       'Gyeongnam', 'Incheon', 'Jeju', 'Jeonbuk', 'Jeonnam',
       'Southern Seoul', 'Ulsan', 'Western Seoul', 'Seoul'], dtype=object)

In [268]:
# remove Southern Seoul, Northern Seoul, Western Seoul, Eastern Seoul
final_df = final_df[~final_df['city'].isin(["Southern Seoul", "Northern Seoul", "Western Seoul", "Eastern Seoul", "Central Seoul"])]

print(final_df["city"].unique())

['Busan' 'Chungbuk' 'Daegu/Gyeongbuk' 'Daejeon/Chungnam' 'Gangwon'
 'Gyeonggi' 'Gyeongnam' 'Incheon' 'Jeju' 'Jeonbuk' 'Jeonnam' 'Ulsan'
 'Seoul']


In [269]:
# check any null values
final_df.isnull().sum()

city                0
year                0
month               0
composition_rate    0
total_unit          0
women_unit          0
men_unit            0
dtype: int64

In [275]:
# round the values to 2 decimal places

final_df["total_unit"] = final_df["total_unit"].astype(float)

final_df["month"] = final_df["month"].astype(int)

final_df["men_unit"] = final_df["men_unit"].astype(float)

final_df["women_unit"] = final_df["women_unit"].astype(float)

final_df["composition_rate"] = final_df["composition_rate"].round(2)

final_df["year"] = final_df["year"].astype(int)


# save it to a csv file
final_df.to_csv('monthly_supply.csv', index=False)

# save as json
final_df.to_json('monthly_supply.json', orient='records')

